This file will be where I scrape the data for all of the pokemon. Here are the sites that I plan to scrape from. This list may be enlargened in the future:
* serebii.net

Webscraping initially will be with the help of ChatGPT

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
#credit ChatGPT for the scrape

url = "https://www.serebii.net/pokemon/nationalpokedex.shtml"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0;Win64) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text, "html.parser")
print(res.status_code)

elements = soup.find("tbody", class_="dextable")
if not elements:
    print("No elements found.")
'''
else:
    for tr in elements:
        print("yes i am here")
        td = tr.find_all("td")
        name = td[1].find("a").text
        types = [x.text for x in td[2].find_all("a")]
        abilities = [x.text for x in td[3].find_all("a")]
        stats = [x.text for x in td[4:10]]
        print("yes i am here")
'''

In [ ]:
print("Name: ", name)
print("Types: ", types)
print("Abilities: ", abilities)
print("Stats: ", stats)

In [ ]:
import pandas as pd

URL = "https://www.serebii.net/pokemon/nationalpokedex.shtml"
dataframes = pd.read_html(URL)
dataframes

In [ ]:
import scrapy
import time
import pandas as pd

class Pokemonspider(scrapy.Spider):
    name = "pokemon"
    start_urls = ["https://www.serebii.net/pokemon/nationalpokedex.shtml"]

    def parse(self, response):
        counter = 0
        mylist = []
        elements = response.xpath("//table[@class='dextable']//tr")
        for row in elements:
            if (counter < 2):
                counter += 1
                continue
                
            cells = row.xpath(".//td")
            if (len(cells) < 3):
                continue
            if cells:
                name = cells[3].xpath(".//a/text()").get()
                types = cells[4].xpath(".//a/@href").getall()
                abilities = cells[5].xpath(".//a/text()").getall()
                stats = cells[6:].xpath(".//text()").getall()
                
                
                
                if (len(types) == 2):
                    type2 = types[1].split('/')[-1]
                    type1 = types[0].split('/')[-1]
                else:
                    type2 = None
                    type1 = types[0].split('/')[-1]
                
                if (len(abilities) == 3):
                    ability1 = abilities[0]
                    ability2 = abilities[1]
                    hiddenability = abilities[2]
                elif (len(abilities) == 2):
                    hiddenability = None
                    ability2 = abilities[1]
                    ability1 = abilities[0]
                else:
                    hiddenability = None
                    ability2 = None
                    ability1 = abilities[0]
                
                
                    
                hp = stats[0]
                atk = stats[1]
                physdef = stats[2]
                spatk = stats[3]
                spdef = stats[4]
                speed = stats[5]
                    
                print(name, type1, type2, ability1, ability2, hiddenability, hp, atk, physdef, spatk, spdef, speed)
                mylist.append({
                    "name": name,
                    "type1": type1,
                    "type2": type2,
                    "ability1": ability1,
                    "ability2": ability2, 
                    "hiddenability": hiddenability,
                    "hp": hp,
                    "atk": atk,
                    "physdef": physdef,
                    "spatk": spatk,
                    "spdef": spdef,
                    "speed": speed,
                    })
                
                
                time.sleep(0.2)
            df = pd.DataFrame(mylist)
            df.to_csv("Pokemon_Data.csv", index=False)
            

print(Pokemonspider.parse)




In [ ]:
import numpy as np
from scrapy.crawler import CrawlerProcess

process = CrawlerProcess()
process.crawl(Pokemonspider)
process.start()

Initial Scrape is done. Now I need to fill in all the cracks that the initial scrape couldn't fill:
Checked for missing pokemon:
* All regional variants (Paldean, Galarian, Alolan)
* All mega evolutions
* Alternate Rotom Forms
* Alternate Oricorio Forms
* Alternate Lycanroc Forms
* Alternate Wishiwashi form
* Alternate Silvally forms
* Alternate Minior Form
* Alternate Greninja Forms
* Alternate Zygarde Forms
* Alternate Deoxys Forms
* Alternate Shaymin forms
* Alternate Aegislash form
* basculin?

for the purposes of this analysis, all regional variants will be treated as completely unique pokemon, as well as all the other forms that are ranked differently by competitive pokemon, which include:
* mega evolutions
* rotom
* oricorio
* lycanroc
* silvally
* greninja
* zygarde
* deoxys
* shaymin

The others (Aegislash, Minior, wishiwashi) are alternate forms that are achieved during battle, and so they won't be included in this scrape. This also includes visual form differences such as spinda and furfrou

the next scrapes will be made by myself, but with most of the aspects copied from the previous scrape (which was generated with chatgpt)

In [1]:
# this scrape will gather all the info from pages where the results are neatly stored in an html table. 
# the other alternate forms are located within their respective parent pages and so the process for scraping them
# will be different

import numpy as np
import scrapy
import time
import pandas as pd
import tqdm
from scrapy.exporters import CsvItemExporter



class PokemonSpider2(scrapy.Spider):
    name = 'missingmons'
    start_urls = ['https://www.serebii.net/sunmoon/alolaforms.shtml', 
                 'https://www.serebii.net/swordshield/galarianforms.shtml', 
                 'https://www.serebii.net/scarletviolet/paldeanforms.shtml', 
                 'https://www.serebii.net/pokedex-xy/megaevolution.shtml']
    
    def start_requests(self):
        for url in self.start_urls:
            if "alola" in url:
                yield scrapy.Request(url=url, callback=self.parse_alola)
            elif "galari" in url:
                yield scrapy.Request(url=url, callback=self.parse_galar)
            elif "paldea" in url:
                yield scrapy.Request(url=url, callback=self.parse_paldea)
            elif "mega" in url:
                yield scrapy.Request(url=url, callback=self.parse_mega)
    
    def parse_alola(self, response):
        myfunc = self.create_data_storer('Alola', response)
        myfunc()
    
    def parse_galar(self, response):
        myfunc = self.create_data_storer('Galar', response)
        myfunc()
    
    def parse_paldea(self, response):
        myfunc = self.create_data_storer('Paldea', response)
        myfunc()
    
    def parse_mega(self, response):
        myfunc = self.create_data_storer('Mega', response)
        myfunc()
        
    
    
    def create_data_storer(self, whichone, response):
        # whichone is can be 'alola', 'galar', 'paldea', 'mega'
        def resultfunction():

            if (whichone == 'Mega'):
                elements = response.xpath("//table[@class='dextable']//tr")
            else:
                elements = response.xpath("//table[@class='tab']//tr")

            print('elements', len(elements))


            counter = 0
            mylist = []

            for row in elements:
                if (counter < 2):
                    counter+=1
                    continue

                cells = row.xpath(".//td")
                print('cells', len(cells))
                if (len(cells) < 12):
                    continue

                if cells:
                    
                    if (whichone == 'Mega'):
                        name = cells[3].xpath(".//a/text()").get()[5:] + '-' + whichone
                    else:
                        name = cells[3].xpath(".//a/text()").get() + '-' + whichone

                    if (whichone == 'Alola'):
                        types = cells[5].xpath(".//a/@href").getall()
                        abilities = cells[4].xpath(".//a/@href").getall()
                    else:
                        types = cells[4].xpath(".//a/@href").getall()
                        abilities = cells[5].xpath(".//a/text()").getall()

                    stats = cells[6:12].xpath(".//text()").getall()



                    if (len(types) == 2):
                        type2 = types[1].split('/')[-1].split('.')[0]
                        type1 = types[0].split('/')[-1].split('.')[0]
                    elif (len(types) == 1):
                        type2 = None
                        type1 = types[0].split('/')[-1].split('.')[0]

                    if (whichone == 'Alola'):
                        #there's such a small amount of missing spaces in this subset that I can go through it manually
                        #that will be put in its own function upon further thought
                        
                        if (len(abilities) == 3):
                            ability1 = abilities[0].split('/')[-1].split('.')[0]
                            ability1 = fix_ability_formatting(ability1)
                            ability2 = abilities[1].split('/')[-1].split('.')[0]
                            ability2 = fix_ability_formatting(ability2)
                            hiddenability = abilities[2].split('/')[-1].split('.')[0]
                            hiddenability = fix_ability_formatting(hiddenability)
                        elif (len(abilities) == 2):
                            hiddenability = None
                            ability2 = abilities[1].split('/')[-1].split('.')[0]
                            ability2 = fix_ability_formatting(ability2)
                            ability1 = abilities[0].split('/')[-1].split('.')[0]
                            ability1 = fix_ability_formatting(ability1)
                        else:
                            hiddenability = None
                            ability2 = None
                            ability1 = abilities[0].split('/')[-1].split('.')[0]
                            ability1 = fix_ability_formatting(ability1)
                    else:
                        if (len(abilities) == 3):
                            ability1 = abilities[0]
                            ability1 = fix_ability_formatting(ability1)
                            ability2 = abilities[1]
                            ability2 = fix_ability_formatting(ability2)
                            hiddenability = abilities[2]
                            hiddenability = fix_ability_formatting(hiddenability)
                        elif (len(abilities) == 2):
                            hiddenability = None
                            ability2 = abilities[1]
                            ability2 = fix_ability_formatting(ability2)
                            ability1 = abilities[0]
                            ability1 = fix_ability_formatting(ability1)
                        else:
                            hiddenability = None
                            ability2 = None
                            ability1 = abilities[0]
                            ability1 = fix_ability_formatting(ability1)



                    hp = stats[0]
                    atk = stats[1]
                    physdef = stats[2]
                    spatk = stats[3]
                    spdef = stats[4]
                    speed = stats[5]

                    print(name, type1, type2, ability1, ability2, hiddenability, hp, atk, physdef, spatk, spdef, speed)
                    mydict = {
                        "name": name,
                        "type1": type1,
                        "type2": type2,
                        "ability1": ability1,
                        "ability2": ability2, 
                        "hiddenability": hiddenability,
                        "hp": hp,
                        "atk": atk,
                        "physdef": physdef,
                        "spatk": spatk,
                        "spdef": spdef,
                        "speed": speed,
                        }
                    mylist.append(mydict)

                    time.sleep(0.2)



            tempdf = pd.DataFrame(mylist)
            tempdf.to_csv('output.csv', mode='a', index=False, header=False)
        return resultfunction
    
    
    
    


print(PokemonSpider2.create_data_storer)

<function PokemonSpider2.create_data_storer at 0x000002534C7E7820>


In [2]:
import numpy as np
from scrapy.crawler import CrawlerProcess

def fix_ability_formatting(ability):
        badthinglist = ['thick', 'surge', 'snow', 'slush', 'sand', 'tangling', 'fur', 
                        'magnet', 'poison', 'cursed', 'lightning', 'rock']
    
        for thing in badthinglist:
            if thing in ability:
                return thing.capitalize() + ' ' + ability[len(thing):].capitalize()
            
        if ' ' in ability:
            return ability[0:ability.find(' ')].capitalize() + ' ' + ability[ability.find(' ')+1:].capitalize()
        
        return ability.capitalize()
    


process = CrawlerProcess()
process.crawl(PokemonSpider2)
process.start()


2023-02-11 19:41:35 [scrapy.utils.log] INFO: Scrapy 2.6.2 started (bot: scrapybot)
2023-02-11 19:41:35 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 22.0.0 (OpenSSL 1.1.1q  5 Jul 2022), cryptography 37.0.1, Platform Windows-10-10.0.19044-SP0
2023-02-11 19:41:35 [scrapy.crawler] INFO: Overridden settings:
{}
2023-02-11 19:41:36 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2023-02-11 19:41:36 [scrapy.extensions.telnet] INFO: Telnet Password: f478bbd510eb6a05
2023-02-11 19:41:36 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2023-02-11 19:41:36 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scr

elements 38
cells 12
Rattata-Alola dark normal Gluttony Hustle Thick Fat 30 56 35 25 35 72
cells 1
cells 12
Raticate-Alola dark normal Gluttony Hustle Thick Fat 75 71 70 40 80 77
cells 1
cells 12
Raichu-Alola electric psychic Surge Surfer None None 60 85 50 95 85 110
cells 1
cells 12
Sandshrew-Alola ice steel Snow Cloak Slush Rush None 50 75 90 10 35 40
cells 1
cells 12
Sandslash-Alola ice steel Snow Cloak Slush Rush None 75 100 120 25 65 65
cells 1
cells 12
Vulpix-Alola ice None Snow Cloak Snow Warning None 38 41 40 50 65 65
cells 1
cells 12
Ninetales-Alola ice fairy Snow Cloak Snow Warning None 73 67 75 81 100 109
cells 1
cells 12
Diglett-Alola ground steel Sand Veil Tangling Hair Sand Force 10 55 30 35 45 90
cells 1
cells 12
Dugtrio-Alola ground steel Sand Veil Tangling Hair Sand Force 35 100 60 50 70 110
cells 1
cells 12
Meowth-Alola dark None Pickup Technician Rattled 40 35 35 50 40 90
cells 1
cells 12
Persian-Alola dark None Fur Coat Technician Rattled 65 60 60 75 65 115
cells 1


cells 1
cells 13
Abomasnow-Mega grass ice Snow Warning None None 90 132 105 132 105 30
cells 1
cells 13
Gallade-Mega psychic fighting Inner Focus None None 68 165 95 65 115 110
cells 1
cells 13
Audino-Mega normal fairy Healer None None 103 60 126 80 126 50
cells 1
cells 13
Diancie-Mega rock fairy Magic Bounce None None 50 160 110 160 110 110


2023-02-11 19:41:57 [scrapy.core.engine] INFO: Closing spider (finished)
2023-02-11 19:41:57 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 993,
 'downloader/request_count': 4,
 'downloader/request_method_count/GET': 4,
 'downloader/response_bytes': 59904,
 'downloader/response_count': 4,
 'downloader/response_status_count/200': 4,
 'elapsed_time_seconds': 20.481855,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2023, 2, 12, 3, 41, 57, 495941),
 'httpcompression/response_bytes': 336951,
 'httpcompression/response_count': 4,
 'log_count/DEBUG': 5,
 'log_count/INFO': 10,
 'response_received_count': 4,
 'scheduler/dequeued': 4,
 'scheduler/dequeued/memory': 4,
 'scheduler/enqueued': 4,
 'scheduler/enqueued/memory': 4,
 'start_time': datetime.datetime(2023, 2, 12, 3, 41, 37, 14086)}
2023-02-11 19:41:57 [scrapy.core.engine] INFO: Spider closed (finished)


cells 1


Given the shortness of the list of mons that I'm missing (13 + the silvallies), I may just put them in manually, since all their information is spread out across like 10 pages that are all formatted in a way more complex way than just the html tables i had before. 
* Rotom *
* Oricorio *
* Lycanroc *
* Silvally *
* Greninja *
* Zygarde *
* Deoxys *
* Shaymin *
* Urshifu

In [220]:
import numpy as np
import pandas as pd

missing_mons = \
    [{'name':'Rotom-Fan', 'type1':'electric', 'type2':'flying', 'ability1':'Levitate', 'ability2':None, 'hiddenability':None,
      'hp':50, 'atk':65, 'physdef':107, 'spatk':105, 'spdef':107, 'speed':86}, 
     {'name':'Rotom-Frost', 'type1':'electric', 'type2':'ice', 'ability1':'Levitate', 'ability2':None, 'hiddenability':None,
      'hp':50, 'atk':65, 'physdef':107, 'spatk':105, 'spdef':107, 'speed':86}, 
     {'name':'Rotom-Heat', 'type1':'electric', 'type2':'fire', 'ability1':'Levitate', 'ability2':None, 'hiddenability':None,
      'hp':50, 'atk':65, 'physdef':107, 'spatk':105, 'spdef':107, 'speed':86}, 
     {'name':'Rotom-Mow', 'type1':'electric', 'type2':'grass', 'ability1':'Levitate', 'ability2':None, 'hiddenability':None,
      'hp':50, 'atk':65, 'physdef':107, 'spatk':105, 'spdef':107, 'speed':86}, 
     {'name':'Rotom-Wash', 'type1':'electric', 'type2':'water', 'ability1':'Levitate', 'ability2':None, 'hiddenability':None,
      'hp':50, 'atk':65, 'physdef':107, 'spatk':105, 'spdef':107, 'speed':86}, 
     \
     {'name':"Oricorio-Pa'u", 'type1':'psychic', 'type2':'flying', 'ability1':'Dancer', 'ability2':None, 'hiddenability':None,
      'hp':75, 'atk':70, 'physdef':70, 'spatk':98, 'spdef':70, 'speed':93}, 
     {'name':'Oricorio-Pom-Pom', 'type1':'electric', 'type2':'flying', 'ability1':'Dancer', 'ability2':None, 'hiddenability':None,
      'hp':75, 'atk':70, 'physdef':70, 'spatk':98, 'spdef':70, 'speed':93}, 
     {'name':'Oricorio-Sensu', 'type1':'ghost', 'type2':'flying', 'ability1':'Dancer', 'ability2':None, 'hiddenability':None,
      'hp':75, 'atk':70, 'physdef':70, 'spatk':98, 'spdef':70, 'speed':93},
     \
     #lycanroc
     {'name':'Lycanroc-Dusk', 'type1':'rock', 'type2':None, 'ability1':'Tough Claws','ability2':None, 'hiddenability':None,
      'hp':75, 'atk':117, 'physdef':65, 'spatk':55, 'spdef':65, 'speed':110},
     {'name':'Lycanroc-Midnight', 'type1':'rock', 'type2':None, 'ability1':'Keen Eye', 'ability2':'Vital Spirit', 'hiddenability':'No Guard',
      'hp':85, 'atk':115, 'physdef':75, 'spatk':55, 'spdef':75, 'speed':82},
     \
     {'name':'Silvally-Bug', 'type1':'bug', 'type2':None, 'ability1':'RKS System', 'ability2':None, 'hiddenability':None,
      'hp':95, 'atk':95, 'physdef':95, 'spatk':95, 'spdef':95, 'speed':95},
     {'name':'Silvally-Dark', 'type1':'dark', 'type2':None, 'ability1':'RKS System', 'ability2':None, 'hiddenability':None,
      'hp':95, 'atk':95, 'physdef':95, 'spatk':95, 'spdef':95, 'speed':95},
     {'name':'Silvally-Dragon', 'type1':'dragon', 'type2':None, 'ability1':'RKS System', 'ability2':None, 'hiddenability':None,
      'hp':95, 'atk':95, 'physdef':95, 'spatk':95, 'spdef':95, 'speed':95},
     {'name':'Silvally-Electric', 'type1':'electric', 'type2':None, 'ability1':'RKS System', 'ability2':None, 'hiddenability':None,
      'hp':95, 'atk':95, 'physdef':95, 'spatk':95, 'spdef':95, 'speed':95},
     {'name':'Silvally-Fairy', 'type1':'fairy', 'type2':None, 'ability1':'RKS System', 'ability2':None, 'hiddenability':None,
      'hp':95, 'atk':95, 'physdef':95, 'spatk':95, 'spdef':95, 'speed':95},
     {'name':'Silvally-Fighting', 'type1':'fighting', 'type2':None, 'ability1':'RKS System', 'ability2':None, 'hiddenability':None,
      'hp':95, 'atk':95, 'physdef':95, 'spatk':95, 'spdef':95, 'speed':95},
     {'name':'Silvally-Fire', 'type1':'fire', 'type2':None, 'ability1':'RKS System', 'ability2':None, 'hiddenability':None,
      'hp':95, 'atk':95, 'physdef':95, 'spatk':95, 'spdef':95, 'speed':95},
     {'name':'Silvally-Flying', 'type1':'flying', 'type2':None, 'ability1':'RKS System', 'ability2':None, 'hiddenability':None,
      'hp':95, 'atk':95, 'physdef':95, 'spatk':95, 'spdef':95, 'speed':95},
     {'name':'Silvally-Ghost', 'type1':'ghost', 'type2':None, 'ability1':'RKS System', 'ability2':None, 'hiddenability':None,
      'hp':95, 'atk':95, 'physdef':95, 'spatk':95, 'spdef':95, 'speed':95},
     {'name':'Silvally-Grass', 'type1':'grass', 'type2':None, 'ability1':'RKS System', 'ability2':None, 'hiddenability':None,
      'hp':95, 'atk':95, 'physdef':95, 'spatk':95, 'spdef':95, 'speed':95},
     {'name':'Silvally-Ground', 'type1':'ground', 'type2':None, 'ability1':'RKS System', 'ability2':None, 'hiddenability':None,
      'hp':95, 'atk':95, 'physdef':95, 'spatk':95, 'spdef':95, 'speed':95},
     {'name':'Silvally-Ice', 'type1':'ice', 'type2':None, 'ability1':'RKS System', 'ability2':None, 'hiddenability':None,
      'hp':95, 'atk':95, 'physdef':95, 'spatk':95, 'spdef':95, 'speed':95},
     {'name':'Silvally-Poison', 'type1':'poison', 'type2':None, 'ability1':'RKS System', 'ability2':None, 'hiddenability':None,
      'hp':95, 'atk':95, 'physdef':95, 'spatk':95, 'spdef':95, 'speed':95},
     {'name':'Silvally-Psychic', 'type1':'psychic', 'type2':None, 'ability1':'RKS System', 'ability2':None, 'hiddenability':None,
      'hp':95, 'atk':95, 'physdef':95, 'spatk':95, 'spdef':95, 'speed':95},
     {'name':'Silvally-Rock', 'type1':'rock', 'type2':None, 'ability1':'RKS System', 'ability2':None, 'hiddenability':None,
      'hp':95, 'atk':95, 'physdef':95, 'spatk':95, 'spdef':95, 'speed':95},
     {'name':'Silvally-Steel', 'type1':'steel', 'type2':None, 'ability1':'RKS System', 'ability2':None, 'hiddenability':None,
      'hp':95, 'atk':95, 'physdef':95, 'spatk':95, 'spdef':95, 'speed':95},
     {'name':'Silvally-Water', 'type1':'water', 'type2':None, 'ability1':'RKS System', 'ability2':None, 'hiddenability':None,
      'hp':95, 'atk':95, 'physdef':95, 'spatk':95, 'spdef':95, 'speed':95},
     \
     {'name':'Greninja-Ash', 'type1':'water', 'type2':'dark', 'ability1':'Battle Bond', 'ability2':None, 'hiddenability':None,
      'hp':72, 'atk':145, 'physdef':67, 'spatk':153, 'spdef':72, 'speed':132},
     \
     {'name':'Zygarde-10%', 'type1':'dragon', 'type2':'ground', 'ability1':'Aura Break', 'ability2':None, 'hiddenability':'Power Construct',
      'hp':54, 'atk':100, 'physdef':71, 'spatk':61, 'spdef':85, 'speed':115},
     {'name':'Zygarde-Complete', 'type1':'dragon', 'type2':'ground', 'ability1':'Power Construct', 'ability2':None, 'hiddenability':None,
      'hp':216, 'atk':100, 'physdef':121, 'spatk':91, 'spdef':95, 'speed':85},
     \
     {'name':'Deoxys-Attack', 'type1':'psychic', 'type2':None, 'ability1':'Pressure', 'ability2':None, 'hiddenability':None,
      'hp':50, 'atk':180, 'physdef':20, 'spatk':180, 'spdef':20, 'speed':150},
     {'name':'Deoxys-Defense', 'type1':'psychic', 'type2':None, 'ability1':'Pressure', 'ability2':None, 'hiddenability':None,
      'hp':50, 'atk':70, 'physdef':160, 'spatk':70, 'spdef':160, 'speed':90},
     {'name':'Deoxys-Speed', 'type1':'psychic', 'type2':None, 'ability1':'Pressure', 'ability2':None, 'hiddenability':None,
      'hp':50, 'atk':95, 'physdef':90, 'spatk':95, 'spdef':90, 'speed':180},
     \
     {'name':'Shaymin-Sky', 'type1':'grass', 'type2':'flying', 'ability1':'Serene Grace', 'ability2':None, 'hiddenability':None,
      'hp':100, 'atk':103, 'physdef':75, 'spatk':120, 'spdef':75, 'speed':127},
     \
     {'name':'Urshifu-Rapid-Strike', 'type1':'fighting', 'type2':'water', 'ability1':'Unseen Fist', 'ability2':None, 'hiddenability':None,
      'hp':100, 'atk':130, 'physdef':100, 'spatk':63, 'spdef':60, 'speed':97},
     \
     {'name':'Zacian-Crowned', 'type1':'fairy', 'type2':'steel', 'ability1':'Intrepid Sword', 'ability2':None, 'hiddenability':None,
      'hp':92, 'atk':150, 'physdef':115, 'spatk':80, 'spdef':115, 'speed':148},
     \
     {'name':'Zamazenta-Crowned', 'type1':'fighting', 'type2':'steel', 'ability1':'Dauntless Shield', 'ability2':None, 'hiddenability':None,
      'hp':92, 'atk':120, 'physdef':140, 'spatk':80, 'spdef':140, 'speed':128},
     \
     {'name':'Kyurem-Black', 'type1':'dragon', 'type2':'ice', 'ability1':'Teravolt', 'ability2':None, 'hiddenability':None,
      'hp':125, 'atk':170, 'physdef':100, 'spatk':120, 'spdef':90, 'speed':95},
     \
     {'name':'Kyurem-White', 'type1':'dragon', 'type2':'ice', 'ability1':'Turboblaze', 'ability2':None, 'hiddenability':None,
      'hp':125, 'atk':120, 'physdef':90, 'spatk':170, 'spdef':100, 'speed':95},
     \
     {'name':'Wormadam-Sandy', 'type1':'bug', 'type2':'ground', 'ability1':'Anticipation', 'ability2':'Overcoat', 'hiddenability':None,
      'hp':60, 'atk':79, 'physdef':105, 'spatk':59, 'spdef':85, 'speed':36},
     {'name':'Wormadam-Trash', 'type1':'bug', 'type2':'steel', 'ability1':'Anticipation', 'ability2':'Overcoat', 'hiddenability':None,
      'hp':60, 'atk':69, 'physdef':95, 'spatk':69, 'spdef':95, 'speed':36},
     \
     {'name':'Groudon-Primal', 'type1':'ground', 'type2':'fire', 'ability1':'Desolate Land', 'ability2':None, 'hiddenability':None,
      'hp':100, 'atk':180, 'physdef':160, 'spatk':150, 'spdef':90, 'speed':90},
     {'name':'Kyogre-Primal', 'type1':'water', 'type2':None, 'ability1':'Primordial Sea', 'ability2':None, 'hiddenability':None,
      'hp':100, 'atk':150, 'physdef':90, 'spatk':180, 'spdef':160, 'speed':90},
     \
     {'name':'Pumpkaboo-Large', 'type1':'ghost', 'type2':'grass', 'ability1':'Pickup', 'ability2':'Frisk', 'hiddenability':'Insomnia',
      'hp':54, 'atk':66, 'physdef':70, 'spatk':44, 'spdef':55, 'speed':46},
     {'name':'Pumpkaboo-Small', 'type1':'ghost', 'type2':'grass', 'ability1':'Pickup', 'ability2':'Frisk', 'hiddenability':'Insomnia',
      'hp':44, 'atk':66, 'physdef':70, 'spatk':44, 'spdef':55, 'speed':56},
     {'name':'Pumpkaboo-Super', 'type1':'ghost', 'type2':'grass', 'ability1':'Pickup', 'ability2':'Frisk', 'hiddenability':'Insomnia',
      'hp':59, 'atk':66, 'physdef':70, 'spatk':44, 'spdef':55, 'speed':41},
     \
     {'name':'Gourgeist-Large', 'type1':'ghost', 'type2':'grass', 'ability1':'Pickup', 'ability2':'Frisk', 'hiddenability':'Insomnia',
      'hp':75, 'atk':95, 'physdef':122, 'spatk':58, 'spdef':75, 'speed':69},
     {'name':'Gourgeist-Small', 'type1':'ghost', 'type2':'grass', 'ability1':'Pickup', 'ability2':'Frisk', 'hiddenability':'Insomnia',
      'hp':55, 'atk':85, 'physdef':122, 'spatk':58, 'spdef':75, 'speed':99},
     {'name':'Gourgeist-Super', 'type1':'ghost', 'type2':'grass', 'ability1':'Pickup', 'ability2':'Frisk', 'hiddenability':'Insomnia',
      'hp':85, 'atk':100, 'physdef':122, 'spatk':58, 'spdef':75, 'speed':54},
     \
     {'name':'Meowstic-F', 'type1':'psychic', 'type2':None, 'ability1':'Keen Eye', 'ability2':'Infiltrator', 'hiddenability':'Competitive',
      'hp':74, 'atk':48, 'physdef':76, 'spatk':83, 'spdef':81, 'speed':104},
     \
     {'name':'Oinkologne-F', 'type1':'normal', 'type2':None, 'ability1':'Aroma Veil', 'ability2':'Gluttony', 'hiddenability':'Thick Fat',
      'hp':115, 'atk':90, 'physdef':70, 'spatk':59, 'spdef':90, 'speed':65},
     \
     {'name':'Indeedee-F', 'type1':'psychic', 'type2':'normal', 'ability1':'Own Tempo', 'ability2':'Synchronize', 'hiddenability':'Psychic Surge',
      'hp':70, 'atk':55, 'physdef':65, 'spatk':95, 'spdef':105, 'speed':85},
     \
     {'name':'Arceus-Bug', 'type1':'bug', 'type2':None, 'ability1':'Multitype', 'ability2':None, 'hiddenability':None,
      'hp':120, 'atk':120, 'physdef':120, 'spatk':120, 'spdef':120, 'speed':120},
     {'name':'Arceus-Dark', 'type1':'dark', 'type2':None, 'ability1':'Multitype', 'ability2':None, 'hiddenability':None,
      'hp':120, 'atk':120, 'physdef':120, 'spatk':120, 'spdef':120, 'speed':120},
     {'name':'Arceus-Dragon', 'type1':'dragon', 'type2':None, 'ability1':'Multitype', 'ability2':None, 'hiddenability':None,
      'hp':120, 'atk':120, 'physdef':120, 'spatk':120, 'spdef':120, 'speed':120},
     {'name':'Arceus-Electric', 'type1':'electric', 'type2':None, 'ability1':'Multitype', 'ability2':None, 'hiddenability':None,
      'hp':120, 'atk':120, 'physdef':120, 'spatk':120, 'spdef':120, 'speed':120},
     {'name':'Arceus-Fairy', 'type1':'fairy', 'type2':None, 'ability1':'Multitype', 'ability2':None, 'hiddenability':None,
      'hp':120, 'atk':120, 'physdef':120, 'spatk':120, 'spdef':120, 'speed':120},
     {'name':'Arceus-Fighting', 'type1':'fighting', 'type2':None, 'ability1':'Multitype', 'ability2':None, 'hiddenability':None,
      'hp':120, 'atk':120, 'physdef':120, 'spatk':120, 'spdef':120, 'speed':120},
     {'name':'Arceus-Fire', 'type1':'fire', 'type2':None, 'ability1':'Multitype', 'ability2':None, 'hiddenability':None,
      'hp':120, 'atk':120, 'physdef':120, 'spatk':120, 'spdef':120, 'speed':120},
     {'name':'Arceus-Flying', 'type1':'flying', 'type2':None, 'ability1':'Multitype', 'ability2':None, 'hiddenability':None,
      'hp':120, 'atk':120, 'physdef':120, 'spatk':120, 'spdef':120, 'speed':120},
     {'name':'Arceus-Ghost', 'type1':'ghost', 'type2':None, 'ability1':'Multitype', 'ability2':None, 'hiddenability':None,
      'hp':120, 'atk':120, 'physdef':120, 'spatk':120, 'spdef':120, 'speed':120},
     {'name':'Arceus-Grass', 'type1':'grass', 'type2':None, 'ability1':'Multitype', 'ability2':None, 'hiddenability':None,
      'hp':120, 'atk':120, 'physdef':120, 'spatk':120, 'spdef':120, 'speed':120},
     {'name':'Arceus-Ground', 'type1':'ground', 'type2':None, 'ability1':'Multitype', 'ability2':None, 'hiddenability':None,
      'hp':120, 'atk':120, 'physdef':120, 'spatk':120, 'spdef':120, 'speed':120},
     {'name':'Arceus-Ice', 'type1':'ice', 'type2':None, 'ability1':'Multitype', 'ability2':None, 'hiddenability':None,
      'hp':120, 'atk':120, 'physdef':120, 'spatk':120, 'spdef':120, 'speed':120},
     {'name':'Arceus-Poison', 'type1':'poison', 'type2':None, 'ability1':'Multitype', 'ability2':None, 'hiddenability':None,
      'hp':120, 'atk':120, 'physdef':120, 'spatk':120, 'spdef':120, 'speed':120},
     {'name':'Arceus-Psychic', 'type1':'psychic', 'type2':None, 'ability1':'Multitype', 'ability2':None, 'hiddenability':None,
      'hp':120, 'atk':120, 'physdef':120, 'spatk':120, 'spdef':120, 'speed':120},
     {'name':'Arceus-Rock', 'type1':'rock', 'type2':None, 'ability1':'Multitype', 'ability2':None, 'hiddenability':None,
      'hp':120, 'atk':120, 'physdef':120, 'spatk':120, 'spdef':120, 'speed':120},
     {'name':'Arceus-Steel', 'type1':'steel', 'type2':None, 'ability1':'Multitype', 'ability2':None, 'hiddenability':None,
      'hp':120, 'atk':120, 'physdef':120, 'spatk':120, 'spdef':120, 'speed':120},
     {'name':'Arceus-Water', 'type1':'water', 'type2':None, 'ability1':'Multitype', 'ability2':None, 'hiddenability':None,
      'hp':120, 'atk':120, 'physdef':120, 'spatk':120, 'spdef':120, 'speed':120},
     \
     {'name':'Squawkabilly-Blue', 'type1':'normal', 'type2':'flying', 'ability1':'Intimidate', 'ability2':'Hustle', 'hiddenability':'Guts',
      'hp':82, 'atk':96, 'physdef':51, 'spatk':45, 'spdef':51, 'speed':92},
     {'name':'Squawkabilly-White', 'type1':'normal', 'type2':'flying', 'ability1':'Intimidate', 'ability2':'Hustle', 'hiddenability':'Sheer Force',
      'hp':82, 'atk':96, 'physdef':51, 'spatk':45, 'spdef':51, 'speed':92},
     {'name':'Squawkabilly-Yellow', 'type1':'normal', 'type2':'flying', 'ability1':'Intimidate', 'ability2':'Hustle', 'hiddenability':'Sheer Force',
      'hp':82, 'atk':96, 'physdef':51, 'spatk':45, 'spdef':51, 'speed':92},
     \
     {'name':'Calyrex-Ice', 'type1':'psychic', 'type2':'ice', 'ability1':'As One (Glastrier)', 'ability2':None, 'hiddenability':None,
      'hp':100, 'atk':165, 'physdef':150, 'spatk':85, 'spdef':130, 'speed':50},
     {'name':'Calyrex-Shadow', 'type1':'psychic', 'type2':'ghost', 'ability1':'As One (Spectrier)', 'ability2':None, 'hiddenability':None,
      'hp':100, 'atk':85, 'physdef':80, 'spatk':165, 'spdef':100, 'speed':150},
     \
     {'name':'Landorus-Therian', 'type1':'ground', 'type2':'flying', 'ability1':'Intimidate', 'ability2':None, 'hiddenability':None,
      'hp':89, 'atk':145, 'physdef':90, 'spatk':105, 'spdef':80, 'speed':91},
     {'name':'Thundurus-Therian', 'type1':'electric', 'type2':'flying', 'ability1':'Volt Absorb', 'ability2':None, 'hiddenability':None,
      'hp':79, 'atk':105, 'physdef':70, 'spatk':145, 'spdef':80, 'speed':101},
     {'name':'Thundurus-Therian', 'type1':'flying', 'type2':None, 'ability1':'Regenerator', 'ability2':None, 'hiddenability':None,
      'hp':79, 'atk':100, 'physdef':80, 'spatk':110, 'spdef':90, 'speed':121},
     \
     {'name':'Necrozma-Dawn-Wings', 'type1':'psychic', 'type2':'ghost', 'ability1':'Prism Armor', 'ability2':None, 'hiddenability':None,
      'hp':97, 'atk':113, 'physdef':109, 'spatk':157, 'spdef':127, 'speed':77},
     {'name':'Necrozma-Dusk-Mane', 'type1':'psychic', 'type2':'steel', 'ability1':'Prism Armor', 'ability2':None, 'hiddenability':None,
      'hp':97, 'atk':157, 'physdef':127, 'spatk':113, 'spdef':109, 'speed':77},
    ]

Variants that must be added (second run-through):
* therians
* necrozma forms

In [221]:
form_df = pd.DataFrame(missing_mons)

form_df['bst'] = form_df['hp'] + form_df['atk'] + form_df['physdef'] + form_df['spatk'] + form_df['spdef'] +\
    form_df['speed']
form_df['alternate'] = True

form_df

,name,type1,type2,ability1,ability2,hiddenability,hp,atk,physdef,spatk,spdef,speed,bst,alternate
0,Rotom-Fan,electric,flying,Levitate,None,None,50,65,107,105,107,86,520,True
1,Rotom-Frost,electric,ice,Levitate,None,None,50,65,107,105,107,86,520,True
2,Rotom-Heat,electric,fire,Levitate,None,None,50,65,107,105,107,86,520,True
3,Rotom-Mow,electric,grass,Levitate,None,None,50,65,107,105,107,86,520,True
4,Rotom-Wash,electric,water,Levitate,None,None,50,65,107,105,107,86,520,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,Landorus-Therian,ground,flying,Intimidate,None,None,89,145,90,105,80,91,600,True
75,Thundurus-Therian,electric,flying,Volt Absorb,None,None,79,105,70,145,80,101,580,True
76,Thundurus-Therian,flying,None,Regenerator,None,None,79,100,80,110,90,121,580,True
77,Necrozma-Dawn-Wings,psychic,ghost,Prism Armor,None,None,97,113,109,157,127,77,680,True


In [ ]:
#form_df.to_csv("Natdex_Data.csv", mode='a', index=False, header=False)

# After correcting the formatting for the mega and regional variant pokemon, I need to rewrite Natdex_Data.csv. I will do that here

Looks like Natdex_Data has some repeats. Paldean Tauros is being repeated, which needs to be fixed. Oricorio was the main issue, because it was the same pokemon twice. It was added in both the additional form_df and in the original scrape

In [134]:
pokemon_df = pd.read_csv('Natdex_Data.csv')
pokemon_df.shape

(1140, 14)

In [135]:
# temp_df is a dataframe that contains everything in pokemon_df minus all the variants that I previously added

drop_list = ['Alolan ', 'Paldean ', 'Galarian ', 'Mega ']
temp_df = pokemon_df[~pokemon_df['name'].str.contains('|'.join(drop_list))]
temp_df = temp_df.query('alternate == False')
temp_df.shape

(1008, 14)

In [211]:
# output_df contains the recently re-scraped regional and mega variants

output_df = pd.read_csv('output.csv', header=None)

# renaming columns to match pokemon_df
columndict = {output:pokemon for output, pokemon in zip(output_df.columns, pokemon_df.columns)}
output_df = output_df.rename(columndict, axis=1)

#dropping duplicates
output_df = output_df.query('not (name == "Darmanitan-Galar" and type2 == "fire")')
output_df = output_df.query('not (name == "Mr. Rime-Galar")')
output_df = output_df.query('not (name == "Sirfetch\'d-Galar")')
output_df = output_df.query('not (name == "Perrserker-Galar")')
output_df = output_df.query('not (name == "Runerigus-Galar")')
output_df = output_df.query('not (name == "Obstagoon-Galar")')
output_df = output_df.query('not (name == "Cursola-Galar")')

# adding in columns to sync with pokemon_df
output_df['bst'] = output_df['hp'] + output_df['atk'] + output_df['physdef'] + output_df['spatk'] + \
    output_df['spdef'] + output_df['speed']
output_df['alternate'] = True


output_df.shape

(89, 14)

In [222]:
final_df = pd.concat([temp_df, output_df, form_df], ignore_index=True).reset_index()

# renaming pokemon to match with pokemon showdown
# if ever new pokemon are added, these will totally mess up everything. That's a small issue (its pretty big)

final_df.iloc[1045, 1] = 'Tauros-Paldea-Combat'
final_df.iloc[1046, 1] = 'Tauros-Paldea-Blaze'
final_df.iloc[1047, 1] = 'Tauros-Paldea-Aqua'
final_df.iloc[717, 1] = 'Zygarde'
final_df.iloc[864, 1] = 'Sirfetch’d'
final_df.iloc[82, 1] = 'Farfetch’d'
final_df.iloc[1031, 1] = 'Farfetch’d-Galar'
final_df.iloc[28, 1] = 'Nidoran-F'
final_df.iloc[31, 1] = 'Nidoran-M'
final_df.iloc[668, 1] = 'Flabébé'

final_df.iloc[1050, 1] = 'Charizard-Mega-X'
final_df.iloc[1062, 1] = 'Mewtwo-Mega-X'
final_df.iloc[1051, 1] = 'Charizard-Mega-Y'
final_df.iloc[1063, 1] = 'Mewtwo-Mega-Y'


final_df

,index,name,type1,type2,ability1,ability2,hiddenability,hp,atk,physdef,spatk,spdef,speed,bst,alternate
0,0,Bulbasaur,grass,poison,Overgrow,Chlorophyll,NaN,45,49,49,65,65,45,318,False
1,1,Ivysaur,grass,poison,Overgrow,Chlorophyll,NaN,60,62,63,80,80,60,405,False
2,2,Venusaur,grass,poison,Overgrow,Chlorophyll,NaN,80,82,83,100,100,80,525,False
3,3,Charmander,fire,NaN,Blaze,Solar Power,NaN,39,52,43,60,50,65,309,False
4,4,Charmeleon,fire,NaN,Blaze,Solar Power,NaN,58,64,58,80,65,80,405,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,1171,Landorus-Therian,ground,flying,Intimidate,None,None,89,145,90,105,80,91,600,True
1172,1172,Thundurus-Therian,electric,flying,Volt Absorb,None,None,79,105,70,145,80,101,580,True
1173,1173,Thundurus-Therian,flying,None,Regenerator,None,None,79,100,80,110,90,121,580,True
1174,1174,Necrozma-Dawn-Wings,psychic,ghost,Prism Armor,None,None,97,113,109,157,127,77,680,True


In [224]:
final_df[final_df.name.str.contains("Charizard")]

,index,name,type1,type2,ability1,ability2,hiddenability,hp,atk,physdef,spatk,spdef,speed,bst,alternate
5,5,Charizard,fire,flying,Blaze,Solar Power,NaN,78,84,78,109,85,100,534,False
1050,1050,Charizard-Mega-X,fire,dragon,Tough Claws,NaN,NaN,78,130,111,130,85,100,634,True
1051,1051,Charizard-Mega-Y,fire,flying,Drought,NaN,NaN,78,104,78,159,115,100,634,True


Finally I think i've gotten it to where I can work simultaneously with this and with the pokemon showdown API. Going to write it to the file now

In [225]:
#final_df.to_csv('Natdex_Data.csv', index=False)

That should be it for this task, now that I've successfully merged the showdown and scraped data in the Showdown_API.ipynb

### Additional Cleaning

There's been some additional errors in the data that I've found, so I'll fix them and then rewrite it to the file here

In [3]:
# formatting issue with the ability Compound Eyes
# alternate-formed pokemon all have the tag of isLegend=False. Fixing this as well

def clean_pokemon_dataset(pokemon_df):
    
    #reformatting Compound Eyes
    pokemon_df['ability1'] = pokemon_df['ability1'].str.replace('Compoundeyes', 'Compound Eyes')
    pokemon_df['ability2'] = pokemon_df['ability2'].str.replace('Compoundeyes', 'Compound Eyes')
    pokemon_df['hiddenability'] = pokemon_df['hiddenability'].str.replace('Compoundeyes', 'Compound Eyes')
    
    #fixing the legendary status of the alternate forms
    index_list = [1055, 1056, 1080, 1081, 1082, 1089, 1100, 1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 
                      1109, 1110, 1111, 1112, 1113, 1114, 1115, 1116, 1119, 1120, 1121, 1122, 1123, 1124, 1125, 
                      1126, 1127, 1128, 1129, 1131, 1132, 1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 
                      1150, 1151, 1152, 1153, 1154, 1155, 1156, 1157, 1158, 1162, 1163, 1164, 1165, 1166, 1167, 
                      1168]
    
    pokemon_df.iloc[index_list, 14] = True
    
    return pokemon_df

In [1]:
import pandas as pd

pokemon_df = pd.read_csv('Natdex_Data.csv')

In [4]:
pokemon_df = clean_pokemon_dataset(pokemon_df)

In [5]:
#pokemon_df.to_csv('Natdex_Data.csv', index=False)